In [26]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ete3

from src.phylogenetic_signal import PagelsLambda
from src.ihmp import get_diffs
from src.greengenes import (
    aggregate_otu_table,
    generate_otu_keys,
    aggregate_tree,
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
data = {name: get_diffs(name) for name in ["ibd", "moms", "t2d"]}

In [8]:
%%bash

# Download greengenes OTUs dataset, if necessary
if ! [ -d greengenes/data/gg_13_5_otus/ ]; then
    if ! [ -f greengenes/data/gg_13_5_otus.tar.gz ]; then
        wget -O greengenes/data/gg_13_5_otus.tar.gz \
            https://gg-sg-web.s3-us-west-2.amazonaws.com/downloads/greengenes_database/gg_13_5/gg_13_5_otus.tar.gz
    fi
    tar -xvf greengenes/data/gg_13_5_otus.tar.gz -C greengenes/data/
fi

gg_13_5_otus/
gg_13_5_otus/otus/
gg_13_5_otus/rep_set/
gg_13_5_otus/rep_set_aligned/
gg_13_5_otus/taxonomy/
gg_13_5_otus/trees/
gg_13_5_otus/trees/._61_otus.tree
gg_13_5_otus/trees/61_otus.tree
gg_13_5_otus/trees/._64_otus.tree
gg_13_5_otus/trees/64_otus.tree
gg_13_5_otus/trees/._67_otus.tree
gg_13_5_otus/trees/67_otus.tree
gg_13_5_otus/trees/._70_otus.tree
gg_13_5_otus/trees/70_otus.tree
gg_13_5_otus/trees/._73_otus.tree
gg_13_5_otus/trees/73_otus.tree
gg_13_5_otus/trees/._76_otus.tree
gg_13_5_otus/trees/76_otus.tree
gg_13_5_otus/trees/._79_otus.tree
gg_13_5_otus/trees/79_otus.tree
gg_13_5_otus/trees/._82_otus.tree
gg_13_5_otus/trees/82_otus.tree
gg_13_5_otus/trees/._85_otus.tree
gg_13_5_otus/trees/85_otus.tree
gg_13_5_otus/trees/._88_otus.tree
gg_13_5_otus/trees/88_otus.tree
gg_13_5_otus/trees/._91_otus.tree
gg_13_5_otus/trees/91_otus.tree
gg_13_5_otus/trees/._94_otus.tree
gg_13_5_otus/trees/94_otus.tree
gg_13_5_otus/trees/._97_otus.tree
gg_13_5_otus/trees/97_otus.tree
gg_13_5_otus/t

In [10]:
results = pd.DataFrame(columns=["dataset", "cutoff", "sample", "lambda"])
# pl = PagelsLambda(
#     tree="/home/phil/phylosig/greengenes/data/gg_13_5_otus_99_annotated.tree",
#     format=1,
#     quoted_node_names=True,
# )

# for cutoff in (99, 97, 94, 91, 88, 85):
#     for dataset, df in data.items():
#         otu_keys = generate_otu_keys(cutoff)
#         otu_table = aggregate_otu_table(df, otu_keys)
#         cov = aggregate_covariance_matrix(df.cov(), otu_keys)
#         cov_df = make_cov_df(cov, otu_table.index)
#         for sample in otu_table.columns:
#             pl = PagelsLambda(otu_table[sample], cov_df)
#             results = results.append(
#                 {
#                     "dataset": dataset,
#                     "cutoff": cutoff,
#                     "sample": sample,
#                     "lambda": pl.lambda_,
#                 },
#                 ignore_index=True,
#             )

MemoryError: Unable to allocate 308. GiB for an array with shape (203452, 203452) and data type float64

In [68]:
# master_tree = ete3.Tree(
#     "/home/phil/phylosig/greengenes/data/gg_13_5_otus_99_annotated.tree",
#     format=1,
#     quoted_node_names=True,
# )
for x in [99, 97, 94, 91, 88]:
    mapping, leaves, tree = generate_otu_keys(x)
    # tree = ete3.Tree(
    #     f"/home/phil/phylosig/greengenes/data/gg_13_5_otus/trees/{x}_otus.tree",
    #     format=1,
    #     quoted_node_names=True,
    # )

    for name, df in data.items():
        otu_table = aggregate_otu_table(df, mapping)

        # Restrict tree to OTUs in the OTU table
        dataset_tree = tree.copy()
        dataset_tree.prune(otu_table.index)

        for sample in otu_table.columns:
            pl = PagelsLambda(otu_table[sample], dataset_tree)
            results = results.append(
                {
                    "dataset": name,
                    "cutoff": x,
                    "sample": sample,
                    "lambda": pl.lam,
                },
                ignore_index=True,
            )
            print(f"{name} {x} {sample} {pl.lam}")

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/phil/miniconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_3405720/3945996517.py", line 15, in <cell line: 6>
    otu_table = aggregate_otu_table(df, mapping)
  File "/home/phil/phylosig/src/greengenes.py", line 94, in aggregate_otu_table
    out_df.loc[otu, :] = np.sum(otu_table.loc[vals, :], axis=0)
  File "/home/phil/miniconda3/lib/python3.9/site-packages/pandas/core/indexing.py", line 925, in __getitem__
    return self._getitem_tuple(key)
  File "/home/phil/miniconda3/lib/python3.9/site-packages/pandas/core/indexing.py", line 1100, in _getitem_tuple
    return self._getitem_lowerdim(tup)
  File "/home/phil/miniconda3/lib/python3.9/site-packages/pandas/core/indexing.py", line 822, in _getitem_lowerdim
    return self._getitem_nested_tuple(tup)
  File "/home/phil/miniconda3/lib/python3.9/site-packages/pandas/core/ind

In [95]:
# First verify that leaves match
tree99 = ete3.Tree(
    f"/home/phil/phylosig/greengenes/data/gg_13_5_otus/trees/{x}_otus.tree",
    format=1,
    quoted_node_names=True,
)

tree99_leafset = set([x.name for x in tree99.get_leaves()])
print("Tree", len(tree99_leafset))

map99, keys99, tree99_clone = generate_otu_keys(99)
keys99_leafset = set(keys99)
print("Keys", len(keys99_leafset))
print("Intersection", len(keys99_leafset & tree99_leafset))

# Then check OTU table aggregation
otu_original = data["ibd"]
otu_aggregated = aggregate_otu_table(otu_original, map99)
print(otu_original.shape)
otu_leafset = set(otu_original.columns)
print("Original", len(otu_leafset))
print("Aggregated", len(otu_aggregated.index))
print("Intersection", len(otu_leafset & keys99_leafset))

In [91]:
# Aggregate rows
out = pd.DataFrame(index=otu_original.index)
for otu, vals in map99.items():
    if vals is None:
        continue  # Skip OTUs which are not the first in their group
    elif otu not in otu_original.columns:
        continue
    else:
        vals_in_table = [val for val in vals if val in otu_original.columns]
        out[otu] = np.sum(otu_original[vals_in_table], axis=1)

out


/tmp/ipykernel_3405720/1491326853.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  out[otu] = np.sum(otu_original[vals_in_table], axis=1)


177228     344726     585220     585227  \
site  patient visit sample                                              
feces 3002    6     CSM5FZ3Z     0.0 -11.396731   0.000000  -0.653629   
              8     CSM5FZ44     0.0   0.000000   0.000000 -13.170841   
              9     CSM5FZ46     0.0  10.833370   0.000000  11.931969   
      3003    5     CSM5FZ4E     0.0   0.000000   0.000000   0.000000   
              6     CSM5FZ4G     0.0   0.000000   0.000000  16.415684   
...                              ...        ...        ...        ...   
      2021    11    MSM5LLIO     0.0   0.000000  10.769351  -3.806857   
      2026    4     MSM5LLIQ     0.0   0.000000   0.000000   0.743845   
      2027    4     MSM5LLIS     0.0   0.000000   0.000000   0.000000   
      2014    9     MSM5ZOJY     0.0   9.949313   0.000000   0.811967   
              11    MSM633FF     0.0   3.650603   0.000000  -2.613284   

                                 186714     300041     875709  323154  \
site  patient visit sample                                              
feces 3002    6     CSM5FZ3Z -10.991272 -10.298141   0.307777     0.0   
              8     CSM5FZ44   0.000000   0.000000 -11.299049     0.0   
              9     CSM5FZ46   0.000000   0.000000   0.000000     0.0   
      3003    5     CSM5FZ4E   0.000000   0.000000   0.000000     0.0   
              6     CSM5FZ4G   0.000000   0.000000   0.000000     0.0   
...                                 ...        ...        ...     ...   
      2021    11    MSM5LLIO   0.000000   0.000000   0.000000     0.0   
      2026    4     MSM5LLIQ   0.000000  11.318954  -0.120572     0.0   
      2027    4     MSM5LLIS   0.000000   0.000000   0.000000     0.0   
      2014    9     MSM5ZOJY   0.000000   9.949313  13.686936     0.0   
              11    MSM633FF   0.000000   3.650603 -13.686936     0.0   

                                 110723     184314  ...     204126     437137  \
site  patient visit sample                          ...                         
feces 3002    6     CSM5FZ3Z -11.396731 -10.991272  ...   0.000000 -10.298141   
              8     CSM5FZ44   0.000000   0.000000  ...   0.000000   0.000000   
              9     CSM5FZ46  11.526507   0.000000  ...   0.000000   0.000000   
      3003    5     CSM5FZ4E   0.000000   0.000000  ...   0.000000   0.000000   
              6     CSM5FZ4G   0.000000   0.000000  ...  12.748925   0.000000   
...                                 ...        ...  ...        ...        ...   
      2021    11    MSM5LLIO   0.000000   0.000000  ...   0.000000   0.000000   
      2026    4     MSM5LLIQ  12.012095   0.000000  ...   0.000000   0.000000   
      2027    4     MSM5LLIS   0.000000   0.000000  ...   0.000000   0.000000   
      2014    9     MSM5ZOJY   0.000000  10.642437  ...   0.000000   0.000000   
              11    MSM633FF   0.000000 -10.642437  ...   0.000000   0.000000   

                                 275061  356148  1083508     444857  \
site  patient visit sample                                            
feces 3002    6     CSM5FZ3Z   0.000000     0.0      0.0  -0.385363   
              8     CSM5FZ44   0.000000     0.0      0.0 -11.704510   
              9     CSM5FZ46   0.000000     0.0      0.0  10.833370   
      3003    5     CSM5FZ4E   0.000000     0.0      0.0   0.000000   
              6     CSM5FZ4G  12.055781     0.0      0.0  10.669504   
...                                 ...     ...      ...        ...   
      2021    11    MSM5LLIO   0.000000     0.0      0.0  11.174809   
      2026    4     MSM5LLIQ   0.000000     0.0      0.0   1.383497   
      2027    4     MSM5LLIS   0.000000     0.0      0.0   0.000000   
      2014    9     MSM5ZOJY   0.000000     0.0      0.0  12.514219   
              11    MSM633FF   0.000000     0.0      0.0 -12.514219   

                                 208479    325977     548587     361108  
site  patient visit sample                                               
fece

In [86]:
map99_intersect = 
otu_original[]

KeyError: "None of [Index(['32272', '423455', '272019'], dtype='object')] are in the [columns]"

In [85]:
otu_original.columns

Index(['1000269', '1008348', '1009894', '1012376', '1017181', '1017413',
       '1019823', '1019878', '102222', '1023075',
       ...
       '964363', '968675', '968954', '971907', '975306', '976470', '979707',
       '988375', '988932', '999046'],
      dtype='object', length=1370)

In [59]:
keys99_leafset - tree99_leafset

set()

In [60]:
tree99_leafset - keys99_leafset

{'4397998',
 '238014',
 '842470',
 '339832',
 '576545',
 '208107',
 '1579251',
 '687676',
 '562968',
 '1024239',
 '332051',
 '4398742',
 '1134379',
 '739683',
 '247875',
 '237266',
 '1097562',
 '635447',
 '3157091',
 '517692',
 '4456378',
 '570089',
 '827191',
 '328826',
 '4417169',
 '4394064',
 '2937838',
 '4329448',
 '291187',
 '3756770',
 '4455991',
 '4434709',
 '359668',
 '4449495',
 '221463',
 '548596',
 '293241',
 '4018595',
 '4423851',
 '4371710',
 '4381311',
 '4356641',
 '658957',
 '1110727',
 '562244',
 '1107467',
 '283715',
 '4387141',
 '1003750',
 '370030',
 '3163524',
 '352914',
 '206106',
 '4183051',
 '310760',
 '571698',
 '251775',
 '771360',
 '4319829',
 '2367577',
 '1783082',
 '2442619',
 '4307556',
 '226880',
 '421839',
 '2028169',
 '689727',
 '540046',
 '323918',
 '4368722',
 '578491',
 '4340358',
 '622900',
 '409759',
 '913174',
 '4364251',
 '236394',
 '702614',
 '879229',
 '2456347',
 '331014',
 '838647',
 '529180',
 '4394145',
 '296511',
 '4463382',
 '4420126',
 '4